### This notebook shows examples of how to use the cyclops.processor API on GEMINI.

## Get all patient encounters from St. Michael's hospital from March 1 2020 to March 15 2020, with vitals and labs.

In [1]:
from cyclops.processors.constants import SMH
from cyclops.query import gemini

QUERY_SAVE_PATH = "/mnt/nfs/project/delirium/_extract/first_models/extract.h5"

# patients = gemini.patients(
#     hospitals=[SMH], from_date="2020-03-01", to_date="2020-03-15"
# )
# patients_vitals = gemini.events(patients=patients, category="vitals")
# patients_labs = gemini.events(patients=patients, category="labs")
# patients_vitals.run()
# patients_labs.run()

# print(f"{len(patients_vitals.data)} rows extracted!")
# print(f"{len(patients_labs.data)} rows extracted!")

# patients_vitals.data.to_hdf(
#     QUERY_SAVE_PATH,
#     key=f"query_demo_vitals",
# )
# patients_labs.data.to_hdf(
#     QUERY_SAVE_PATH,
#     key=f"query_demo_labs",
# )

2022-04-11 14:12:27,490 INFO cyclops.config  - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-11 14:12:29,093 INFO cyclops.query.utils - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-11 14:12:29,101 INFO cyclops.utils.profile - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-11 14:12:29,105 INFO cyclops.orm     - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-11 14:12:32,507 INFO cyclops.orm     - Database setup, ready to run queries!


## Process queried data into features. (Static aggregation + no imputation).

In [2]:
import pandas as pd

from cyclops.processor import Aggregator, Imputer, featurize

vitals_data = pd.read_hdf(QUERY_SAVE_PATH, key=f"query_demo_vitals")
labs_data = pd.read_hdf(QUERY_SAVE_PATH, key=f"query_demo_labs")

feature_handler = featurize(
    data=[labs_data, vitals_data],
    imputer=Imputer(strategy="none"),
    aggregator=Aggregator(strategy="static", range_=24, window=24),
)

2022-04-11 14:12:32,571 INFO cyclops.processors.base - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-11 14:12:32,577 INFO cyclops.processors.admin - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-11 14:12:34,141 INFO cyclops.processors.feature_handler - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-11 14:12:34,151 INFO cyclops.processors.labs - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-11 14:12:34,157 INFO cyclops.processors.vitals - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-11 14:12:35,751 INFO cyclops.processors.base - Processing raw admin data...
2022-04-11 14:12:35,758 INFO cyclops.processors.base - # samples: 16243, # encounters: 156
2022-04-11 14:12:35,760 INFO cyclops.processors.admin - # admin features: 2, # encounters: 156
2022-04-11 14:12:35,864 INFO cyclops.utils.profile - Finished executing function process in 0.112638 s
2022-04-11 14:12:35,869 INFO cycl

## Inspect feature handler, and check features.

In [3]:
print(feature_handler.get_numerical_features())
print(feature_handler.get_categorical_features())

['age', 'potassium', 'pt', 'sodium', 'urine sodium', 'urine specific gravity', 'bicarbonate', 'blood urea nitrogen', 'vitamin b12', 'white blood cell count', 'high sensitivity troponin', 'vitamin d', 'hba1c', 'lymphocyte', 'neutrophils', 'albumin', 'aptt', 'bilirubin', 'calcium', 'calcium, ionized', 'creatinine', 'esr', 'serum alcohol', 'ferritin', 'fibrinogen', 'glucose fasting', 'glucose point of care', 'glucose random', 'hematocrit', 'hemoglobin', 'crp', 'inr', 'ketone', 'lactate arterial', 'lactate venous', 'mean cell volume', 'arterial paco2', 'venous pco2', 'arterial ph', 'venous ph', 'platelet count', 'arterial pao2', 'CAM', 'Diastolic BP', 'Heart Rate', 'respiratory rate', 'Systolic BP', 'Temperature', 'weight']
['sex', 'fio2']


## Look at data quality report (patient statics + labs + vitals)

In [4]:
from evidently import ColumnMapping
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataQualityTab

dashboard = Dashboard(tabs=[DataQualityTab()])
column_mapping = ColumnMapping(
    numerical_features=feature_handler.get_numerical_features(),
    categorical_features=feature_handler.get_categorical_features(),
)
dashboard.calculate(
    reference_data=feature_handler.features,
    current_data=None,
    column_mapping=column_mapping,
)
dashboard.save(f"data_processed.html")